# Import Modules

In [1]:
%%capture
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

sys.path.insert(
    0,
    os.path.join(
        os.environ["PROJ_irox"],
        "scripts/01_Michal_OER_Plot_Script"))

sys.path.insert(
    0,
    os.path.join(
        os.environ["PROJ_irox"],
        "data"))

from an_data_processing import load_df
from sc_OER_contour_plot_IrOx_vers2 import create_OER_plots

###########################################################

# Python Modules
import numpy as np
import pandas as pd

import plotly as py
import plotly.graph_objs as go

import colorlover as cl
from IPython.display import HTML

# My Modules
from orr_reaction.orr_fed_plot import ORR_Free_E_Plot

# Project Data
from proj_data_irox import (
    surface_energies,
    smart_format_dict_volcano,

    color_palettes,
    system_color_map,

    max_surf_e,
    min_surf_e,
    data_dir,
    )

smart_format_dict = smart_format_dict_volcano

Divisor is Energy class instance!!!
Divisor is Energy class instance!!!


# Script Inputs

In [3]:
bias=0.

In [4]:
annotation_placement_dict = {
    "100 , O-2 OH-0 , IrO3 rutile-like": [(26, 10)],
    "100 , O-2 OH-2 , IrO3 rutile-like": [(26, 0)],
    "100 , O-4 OH-0 , IrO3 rutile-like": [(22, -22)],

    "110 , h covered , IrO3 rutile-like": [(-15, 15)],
    "110 , o covered , IrO3 rutile-like": [(20, 10)],
   
#     "100 , h covered , IrO2": [(-10, 10)],
#     "100 , h covered , IrO3": [(-10, 10)],
#     "100 , o covered , IrO2": [(-10, 10)],
#     "100 , o covered , IrO3": [(-10, 10)],

    "110 , h covered , IrO2": [(4, 14)],
#     "110 , o covered , IrO2": [(-10, 10)],
    
    "110 , h covered , IrO3": [(4, -22)],
    "110 , o covered , IrO3": [(28, 2)],
    "111 , o covered , IrO3": [(0, -24)],
    "211 , h covered , IrO3": [(-2, -10)],
    "211 , o covered , IrO3": [(8, 16)],
    }

ignore_data_point_list = [
    # "100 , O-2 OH-0 , IrO3 rutile-like",
    # "100 , O-2 OH-2 , IrO3 rutile-like",
    # "100 , O-4 OH-0 , IrO3 rutile-like",

    # "110 , h covered , IrO3 rutile-like",
    # "110 , o covered , IrO3 rutile-like",

    "110 , h covered , IrO2",
    # "110 , o covered , IrO2",
    "100 , h covered , IrO2",
    # "100 , o covered , IrO2",

    # "110 , h covered , IrO3",
    # "110 , o covered , IrO3",
    # "100 , o covered , IrO3",
    "100 , h covered , IrO3",
    # "111 , o covered , IrO3",
    "211 , h covered , IrO3",
    # "211 , o covered , IrO3",
    ]

# Load Data

In [5]:
# dataframe_dir = os.path.join(
#     os.environ["PROJ_irox"],
#     "data",
#     )

df_pourbaix, df_ads, df_surf = load_df(
    from_file=True,
    root_dir=data_dir,
    data_dir=data_dir,
    file_name="df_master.pickle"
    )

df_m = df_ads
df_m = df_m[[not i for i in pd.isna(df_m["elec_energy"].tolist())]]

df_m["name_i"] = df_m["name_i"].str.replace("_", " ")
df_m["name_i"] = df_m["name_i"].str.replace("|", ",")

Attempting to load df from pickle


# Methods

In [6]:
def make_series_entry_4_Michal_sc(
    ORR_series,

    size = "8",
    color1 = "black",
    color2 = "green",
    symbol = "s",
    ):
    """
    """
    series_i = ORR_series

    entry_list = []

    energy_list = series_i.energy_lst
    energy_list = energy_list[1:-1]

    entry_list.extend(energy_list)
    entry_list.append(series_i.overpotential_OER)

    entry_list.append(series_i.opt_name)
    entry_list.append(color1)

    tmp = [0.0, 0.08, 1.5]
    entry_list.extend(tmp)

    entry_list.append(size)
    entry_list.append(color2)
    entry_list.append(symbol)
    
    # TEMP
    entry_list.append(series_i.properties)
    
    return(entry_list)

def color_scale_interp(
    input_num,
    max_num,
    min_num,
    color_mesh_size=80,
    hex_mode=True,
    ):
    """
    """
#     cl.scales["8"]["seq"]["Purples"]

    black_white_cs = [
        'rgb(0,0,0)',
        'rgb(255,255,255)',
        ]

    black_red_cs = [
        'rgb(0,0,0)',
        'rgb(255,0,0)',
        ]
    
    color_scale_i = black_red_cs

    color_scale_i = cl.scales["8"]["seq"]["Greys"][::-1]
#     color_scale_i = cl.scales["8"]["seq"]["Purples"]
#     color_scale_i = cl.scales['3']['div']['RdYlBu']
    
    color_scale = cl.interp(
        color_scale_i,
        color_mesh_size,
        )

    color_scale = cl.to_rgb(color_scale)
    
    # Converting RGB string representatino to tuple
    color_scale = cl.to_numeric(color_scale)

    input_norm = ((input_num - min_num)/ (max_num - min_num))
    
    cs_index = round(input_norm * len(color_scale)) - 1
    if cs_index == -1:
        cs_index = 0
        
    color_out = color_scale[cs_index]
    
    if hex_mode:
        def rgb_to_hex(rgb_tuple):
            """
            """
            r = int(rgb_tuple[0])
            g = int(rgb_tuple[1])
            b = int(rgb_tuple[2])

            def clamp(x): 
              return max(0, min(x, 255))

            hex_rep = "#{0:02x}{1:02x}{2:02x}".format(clamp(r), clamp(g), clamp(b))
            return(hex_rep)

        color_out = rgb_to_hex(color_out)

    
    return(color_out)

# Plotting

In [7]:
prop_name_list = [
    'bulk_system',
    'coverage',
    'coverage_type',
    'facet',
    ]

ORR_PLT = ORR_Free_E_Plot(
    free_energy_df=None,
    state_title="adsorbate",
    free_e_title="ads_e",  
    bias=bias,
    color_list=None,   
    show_H_e_pairs_annotations=True,
    show_legend=True,
    rxn_type="OER",
    )

grouped = df_m.groupby(["facet", "coverage_type", "bulk_system"])
for i_ind, (name, group) in enumerate(grouped):
    df_i = group

    prop_list_i = {}
    for prop_i in prop_name_list:    
        first_value = df_i[prop_i].tolist()[0]
        all_same_value = all([True if i == first_value else False for i in df_i[prop_i].tolist()])

        if all_same_value:
            prop_list_i[prop_i] = first_value
        else:
            prop_list_i[prop_i] = None

    if not any([np.isnan(i) for i in df_i.elec_energy.tolist()]):        
        ORR_PLT.add_series(
            df_i,
            plot_mode="all",
            opt_name=df_i["name_i"].tolist()[0],
            overpotential_type="OER",
            system_properties=prop_list_i,
            )

Not the correct # of steps for H2O2
Not the correct # of steps for H2O2
Not the correct # of steps for H2O2
Not the correct # of steps for H2O2
Not the correct # of steps for H2O2
Not the correct # of steps for H2O2
Not the correct # of steps for H2O2
Not the correct # of steps for H2O2
Not the correct # of steps for H2O2
Not the correct # of steps for H2O2
Not the correct # of steps for H2O2
Not the correct # of steps for H2O2
Not the correct # of steps for H2O2
Not the correct # of steps for H2O2


# Converting ORR instance into form that can be put into Michals script

In [8]:
calc_systems = []
for series_i in ORR_PLT.series_list:

    # **************************************************
#     ads_e = series_i.properties['ads_e']
    bulk_system = series_i.properties['bulk_system']
    coverage = series_i.properties['coverage']
    coverage_type = series_i.properties['coverage_type']
    facet = series_i.properties['facet']
#     job_type = series_i.properties['job_type']
    # **************************************************

    format_dict = {}
    for key_i, value_i in series_i.properties.items():
        for format_i in smart_format_dict:
            if list(format_i[0])[0] == key_i:
                if list(format_i[0].values())[0] == value_i:
                    format_dict.update(format_i[1])

    color_i = system_color_map[bulk_system + "_" + facet]
    surf_e_i = surface_energies[bulk_system + "_" + facet]
    color_2 = color_scale_interp(surf_e_i, max_surf_e, min_surf_e)

    entry_i = make_series_entry_4_Michal_sc(
        series_i,
        size = "8",
        color1 = color_2,
        color2 = color_i,
        symbol = format_dict["symbol"],
        )

    calc_systems.append(entry_i)

# Collecting systems for plotting

In [9]:
calc_systems_final = []
for ind_i, system_i in enumerate(calc_systems):
#     print(system_i[4])
    if system_i[4] not in ignore_data_point_list:
        calc_systems_final.append(system_i)

# Ordering the systems list by the bulk material (was disordered before)

iro2_list = []
iro3_list = []
iro3_rl_list = []

for ind_i, system_i in enumerate(calc_systems_final):
    if "IrO3 rutile-like" in system_i[4]:
        iro3_rl_list.append(system_i)
    elif "IrO2" in system_i[4]:
        iro2_list.append(system_i)
    elif "IrO3" in system_i[4]:
        iro3_list.append(system_i)

# print(len(iro2_list) + len(iro3_list) + len(iro3_rl_list))

calc_systems_final = iro2_list + iro3_list + iro3_rl_list

In [10]:
create_OER_plots(
    calc_systems_final,
    annotation_placement_dict=annotation_placement_dict,
    plots_folder=os.path.join(
        os.environ["PROJ_irox"],
        "pl_master_plots/oer_plots",
        ),
#     "../pl_master_plots/oer_plots",  
    )

<Figure size 1838.11x1136.01 with 0 Axes>